In [2]:
import numpy as np
'''
訓練データと正解ラベル
'''
train = np.array([[0, 0],
                  [0, 1],
                  [1, 0],
                  [1, 1],])
label = np.array([[0],
                  [1],
                  [1],
                  [0],])

'''
モデルの定義
'''
import tensorflow as tf

class MLP(tf.keras.Model):
    '''
    多層パーセプトロン
    Attributes:
        l1(Dense): 隠れ層
        l2(Dense): 出力層
    '''
    
    def __init__(self, input_dim, hidden_dim, output_dim):
        '''
        モデルの初期化を行う
        Parameters:
            input_dim(int): 出力する1データあたりの値の形状
            hidden_dim(int): 隠れ層のニューロン数
            outpu_dim(int): 出力層のニューロン数
        '''
        super(MLP, self).__init__() #スーパークラスの__init__()を実行
        
        #隠れ層
        self.l1 = tf.keras.layers.Dense(
            units = hidden_dim, #ニューロンのサイズ(数)
            input_dim = input_dim, #入力データの形状
            activation = 'sigmoid') #活性化関数=sigmoid
        
        #出力層
        self.l2 = tf.keras.layers.Dense(
            units = output_dim,
            activation = 'sigmoid')
    
    @tf.function
    def call(self, x, training = None):
            '''モデルのインスタンスからコールバックされる関数
            
            MLPの順伝播処理を行う
            
            Parameters:
                x(ndarray(float32)): 訓練データ、or 検証データ
                training(bool): 訓練True, 検証False
                
            Returns(float32): 出力層からの出力値
            '''
            h = self.l1(x) #隠れ層の出力
            y = self.l2(h) #出力層の出力
            return y

#入力層2ニューロン、隠れ層2ニューロン、出力層1ニューロンのモデルを生成
model = MLP(2, 2, 1)

In [4]:
'''
3. 損失関数とオプティマイザーの生成
'''
#バイナリ用のクロスエントロピー誤差のオブジェクトを生成
loss_fn = tf.keras.losses.BinaryCrossentropy()

#勾配降下アルゴリズムを使用するオプティマイザーを生成
optimizer = tf.keras.optimizers.SGD(learning_rate = 0.5)
#学習速度を上げる＝パラメータの更新値を大きくする

In [5]:
'''
4. 勾配降下アルゴリズムによるパラメータの更新処理
'''
@tf.function
def train_step(x, t):
        '''バックプロパゲーションによるパラメータ更新を行う
        
        Parameters: x(ndarray(float32)): 訓練データ
                              t(ndarray(float32)): 正解ラベル
        
        Returns: MLPの出力と正解ラベルのクロスエントロピー誤差
        '''
        
        #自動微分による購買計算のための操作を記録するブロおく
        with tf.GradientTape() as tape:
                predictions = model(x) #モデルに入力して順伝播の出力値を取得
                pred_loss = loss_fn(t, predictions) #出力値と正解ラベルの誤差を取得
                
        #tapeに記録された操作を使用して誤差の勾配を計算
        gradients = tape.gradient(
                pred_loss, #現在の誤差
                model.trainable_variables) #更新可能なバイアス、重みのリストを取得
        
        #勾配降下法の更新式を適用してバイアス、重みを更新
        optimizer.apply_gradients(
                zip(gradients, #取得済みの勾配
                       model.trainable_variables #更新可能なバイアス、重みのリスト
                       ))
        
        return pred_loss 

In [6]:
'''
モデルを使用して学習する
'''
#エポック数
epochs = 4000

#学習を行う
for epoch in range(epochs):
        #1epochごとの損失を保持する変数
        epoch_loss = 0.
        
        #データをモデルに入力し、バイアス、重みを更新して誤差を取得
        loss = train_step(train, label)
        epoch_loss = epoch_loss + loss.numpy()
        
        #1000epochごとに結果を出力
        if (epoch + 1) % 1000 == 0:
                print(f'{epoch + 1}: {epoch_loss}')

model.summary()

2022-08-30 17:09:13.008874: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-30 17:09:13.011388: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


1000: 0.369698703289032
2000: 0.3547428250312805
3000: 0.3513994514942169
4000: 0.3499673902988434
Model: "mlp"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  6         
_________________________________________________________________
dense_1 (Dense)              multiple                  3         
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


In [7]:
'''
6. モデルを評価する
'''
#学習済みのMLPの出力
print(model(train))

#学習した重み・バイアスを使って、XORゲートの出力を表示
#MLPの出力が0.5以上であれば1、そうでなければ0を返す
print(tf.cast(
            (model(train) >= 0.5), tf.int32))

tf.Tensor(
[[0.00350738]
 [0.4986658 ]
 [0.9960474 ]
 [0.5017097 ]], shape=(4, 1), dtype=float32)
tf.Tensor(
[[0]
 [0]
 [1]
 [1]], shape=(4, 1), dtype=int32)


In [14]:
import tensorflow.keras
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import SGD

#XORゲートの入力値、0と1の組み合わせの行列(4*2)
X = np.array([
    [0, 0], [0, 1], [1, 0], [1, 1]
])

#XORゲートの出力、正解ラベルの行列(4*1)、0はFalse、1はTrue
T = np.array([
    [0], [1], [1], [0]
])